In [1]:
import pandas as pd
import json
import requests
from pprint import pprint

# Запрос для Яндекс Метрики
token = #'В СТРОКЕ УКАЗАТЬ СВОЙ РАБОЧИЙ ТОКЕН'
headers = {'Authorization': 'OAuth'+token}
params = {'metrics': 'ym:s:visits',
         'dimensions': 'ym:s:hour,ym:s:lastsignTrafficSource,ym:s:isRobot',
         'date1': '2016-01-01',
         'date2': '2019-07-01',
         'ids': 30177909,
         'accuracy': 'full',
         'limit': 100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data = response.json()

# Формула для перебора данных из запроса
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts=[]
    dimensions_list=metrika_data['query']['dimensions']
    metrics_list=metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d={}
        for i, dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]]=dimension['name']
        for i, metric in enumerate(data_item['metrics']):
            d[metrics_list[i]]=metric
        list_of_dicts.append(d)
    return list_of_dicts


# Перебрал запрос, на основе этих данных, создал DataFrame — metrika_df
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
metrika_df = pd.DataFrame(metrika_list_of_dicts)

# Перевел столбец с визитами в целочисленный дата тип
metrika_df['ym:s:visits'] = (metrika_df['ym:s:visits']).astype(int)

# Убрал пустые значения в столбце ym:s:lastsignTrafficSource
metrika_df = metrika_df.dropna()

# Убрал ym:s из названия столбцов
metrika_df.columns = [col.replace('ym:s:', '') for col in metrika_df.columns]

# Фильтр DataFrame metrika_df по столбцу isRobot, значения — Robots. Новый DataFrame — isRobots_df
isRobots_df = metrika_df[metrika_df['isRobot']=='Robots']

# Посчитал общее количество визитов роботов
All_Robot_visits = isRobots_df['visits'].sum()


# Процент визитов роботов по часами к общему количеству визитов роботов
Visit_Robot_by_the_Hour = ((isRobots_df.groupby('hour')['visits'].sum()) / All_Robot_visits) * 100
for i in range(len(Visit_Robot_by_the_Hour)):
    Visit_Robot_by_the_Hour[i] = "%.2f" % Visit_Robot_by_the_Hour[i]

# Процент визитов роботов, в зависимости от источника к общему количеству визитов роботов
Visit_Robot_by_the_Source = ((isRobots_df.groupby('lastsignTrafficSource')['visits'].sum()) / All_Robot_visits) * 100
for i in range(len(Visit_Robot_by_the_Source)):
    Visit_Robot_by_the_Source[i] = "%.2f" % Visit_Robot_by_the_Source[i]


# Проверка гипотиз:

# №1 — процент визитов роботов по часами к общему количеству визитов роботов:
display(Visit_Robot_by_the_Hour)
# Вывод: с 10:00 до 17:00 количество визитов роботов увеличивается на 1-2%, в зависимости от другого времени. Пик — в 11:00

# №2 — процент визитов роботов, в зависимости от источника, к общему количеству визитов роботов:
display(Visit_Robot_by_the_Source)
# Вывод: большая часть трафика из Директа — 66%, из поиска — 31%. У остальных источников — менее 2%


hour
00:00    3.45
01:00    3.01
02:00    3.81
03:00    2.39
04:00    3.12
05:00    3.34
06:00    2.61
07:00    2.83
08:00    3.45
09:00    3.63
10:00    5.08
11:00    6.49
12:00    5.19
13:00    5.30
14:00    5.66
15:00    5.51
16:00    5.91
17:00    5.08
18:00    4.32
19:00    4.61
20:00    4.06
21:00    4.46
22:00    3.52
23:00    3.19
Name: visits, dtype: float64

lastsignTrafficSource
Direct traffic            66.23
Internal traffic           0.29
Link traffic               1.12
Search engine traffic     30.69
Social network traffic     1.67
Name: visits, dtype: float64